# 3.3 线性回归的简洁实现

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.3.1


## 3.3.1 生成数据集

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)# 加入噪音
print("features:\n",features[:5])
print("labels:\n",labels[:5])

features:
 tensor([[ 0.7052, -0.2788],
        [ 1.6032, -0.6295],
        [-0.7177,  0.4649],
        [-0.4881, -0.5710],
        [ 1.2327,  0.7881]])
labels:
 tensor([6.5668, 9.5614, 1.1909, 5.1758, 3.9733])


## 3.3.2 读取数据

In [3]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)
print(dataset[:5])

(tensor([[ 0.7052, -0.2788],
        [ 1.6032, -0.6295],
        [-0.7177,  0.4649],
        [-0.4881, -0.5710],
        [ 1.2327,  0.7881]]), tensor([6.5668, 9.5614, 1.1909, 5.1758, 3.9733]))


In [4]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-0.0084,  0.5590],
        [ 0.9131,  0.8227],
        [-0.6157, -0.0106],
        [-1.2184, -0.7417],
        [-0.7318,  1.8391],
        [ 0.1242, -0.7889],
        [ 1.5023,  0.0376],
        [-0.9632,  1.5561],
        [ 0.0307,  0.6322],
        [-0.5599,  0.0796]]) 
 tensor([ 2.2854,  3.2280,  3.0065,  4.2889, -3.5170,  7.1413,  7.0796, -3.0404,
         2.0919,  2.8032])


## 3.3.3 定义模型

In [5]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    #定义向前传播
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.5347, 0.7057]], requires_grad=True)
Parameter containing:
tensor([0.6873], requires_grad=True)


## 3.3.4 初始化模型参数

In [8]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [21]:
for name, param in net.named_parameters():
    print(name, param)
    print(name, param.shape)

linear.weight Parameter containing:
tensor([[ 2.0007, -3.3996]], requires_grad=True)
linear.weight torch.Size([1, 2])
linear.bias Parameter containing:
tensor([4.2010], requires_grad=True)
linear.bias torch.Size([1])


## 3.3.5 定义损失函数

In [10]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [13]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [14]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000171
epoch 2, loss: 0.000047
epoch 3, loss: 0.000108


In [15]:
dense = net[0]
print(net)
print(dense)
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)
[2, -3.4] tensor([[ 2.0007, -3.3996]])
4.2 tensor([4.2010])
